In [1]:
import pandas as pd
import os
import numpy as np

from lmu_torch import LMUCell
import torch
from tqdm import tqdm

In [2]:
def load_data(folder):
    X = []
    Y = []
    for fname in os.listdir(folder):
        df = pd.read_csv(f'{folder}/{fname}', skiprows=28)[:-1]
        x = df[[
            'angle_sin', 'angle_cos', 'angleD', 'position', 
            'positionD', 'target_equilibrium', 'target_position'
        ]]
        y = df[['Q']]
        X.append(x)
        Y.append(y)
    X = np.array(X)
    Y = np.array(Y)
    return X, Y

In [3]:
class Model(torch.nn.Module):
    """ A simple model for the psMNIST dataset consisting of a single LMU layer and a single dense classifier """

    def __init__(self, input_size, output_size, hidden_size, memory_size, theta, learn_a = False, learn_b = False):
        super(Model, self).__init__()
        self.lmu = LMUCell(input_size, hidden_size, memory_size, theta, learn_a, learn_b)
        self.classifier = torch.nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out = []
        h_0 = torch.zeros(x.shape[0], self.lmu.hidden_size)
        m_0 = torch.zeros(x.shape[0], self.lmu.memory_size)
        state = (h_0, m_0)
        for t in range(x.shape[1]):
            state = self.lmu(x[:,t,:], state) # [batch_size, hidden_size]
            output = self.classifier(state[0])
            out.append(output) # [batch_size, output_size]
        return torch.stack(out, dim=1) # [batch_size, seq_len, output_size]

In [4]:
folder = 'DG-27s-and-1s500ms-noisy-u/Recordings/Train/Train-1s500ms'
x_train, y_train = load_data(folder)
print(x_train.shape, y_train.shape)

folder = 'DG-27s-and-1s500ms-noisy-u/Recordings/Test/Test-1s500ms'
x_test, y_test = load_data(folder)
print(x_test.shape, y_test.shape)

(4000, 75, 7) (4000, 75, 1)
(500, 75, 7) (500, 75, 1)


In [5]:
model = Model(7, 1, 16, 16, 4)
optimizer = torch.optim.Adam(model.parameters())
loss = torch.nn.MSELoss()

In [6]:
batch_size = 64
n_batches = x_train.shape[0] // batch_size

for epoch in range(500):

    # eval
    if epoch % 50 == 0:
        loss_train = []
        for batch_idx in range(n_batches-1):
            model.eval()
            x = x_train[batch_idx*batch_size:(batch_idx+1)*batch_size]
            y = y_train[batch_idx*batch_size:(batch_idx+1)*batch_size]
            x = torch.tensor(x, dtype=torch.float32)
            y = torch.tensor(y, dtype=torch.float32)
            ypr = model(x)
            loss_train.append(loss(ypr, y).item())
        loss_test = []
        for batch_idx in range(x_test.shape[0] // batch_size):
            model.eval()
            x = x_test[batch_idx*batch_size:(batch_idx+1)*batch_size]
            y = y_test[batch_idx*batch_size:(batch_idx+1)*batch_size]
            x = torch.tensor(x, dtype=torch.float32)
            y = torch.tensor(y, dtype=torch.float32)
            ypr = model(x)
            loss_test.append(loss(ypr, y).item())
        print(epoch, 'train', np.array(loss_train).mean(), 'test', np.array(loss_test).mean())

    # train
    epoch_loss = []
    model.train()
    for batch_idx in range(n_batches-1):
        x = x_train[batch_idx*batch_size:(batch_idx+1)*batch_size]
        y = y_train[batch_idx*batch_size:(batch_idx+1)*batch_size]
        
        x = torch.tensor(x, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.float32)

        optimizer.zero_grad()
        y_pred = model(x)
        l = loss(y_pred, y)
        l.backward()
        optimizer.step()

        epoch_loss.append(l.item())

    # log training loss
    avg_epoch_loss = np.array(epoch_loss).mean()
    if epoch % 10 == 0:
        print(epoch, avg_epoch_loss)
    else:
        print(epoch, avg_epoch_loss, end='\r')

0 train 1.0198590657750115 test 1.0027810675757272
0 0.6348884950895779
10 0.03771992956028615
20 0.022867461483253807
30 0.017467555742649757
40 0.015708216633953033
50 train 0.015048205623494798 test 0.015310620090791158
50 0.0150655366205534
60 0.014666089268981433
70 0.014322236836811558
80 0.013989605543921228
90 0.013637013519640829
100 train 0.013298574331231782 test 0.013720575054841382
100 0.013252442763721357
110 0.012889095750011389
120 0.012610604421647846
130 0.012400065518182816
140 0.012226454165504604
150 train 0.012119033679243971 test 0.0126453572884202
150 0.012076566560712994
160 0.011944944710760828
170 0.011828989309609914
180 0.011726906492573316
190 0.011636394137119667
200 train 0.011617172891121418 test 0.012121343719107764
200 0.011555128471284617
210 0.011481443481122861
220 0.011414455036159421
230 0.011353715429784821
240 0.011298758496881509
250 train 0.011310072188250354 test 0.01179641272340502
250 0.011248878096459343
260 0.011203200938027413
270 0.011

In [8]:
model_name = 'mpc_models/lmu_7-1-16-16-4.pt'
torch.save(model.state_dict(), model_name)